**Project 2 Phase 2**<br>
Group 8
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Uros</td>
    <td style = "text-align: left">Zivanovic</td>
    <td style = "text-align: left">12032271</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Maximilian</td>
    <td style = "text-align: left">Hofmann</td>
    <td style = "text-align: left">12019987</td>
  </tr>
  
</table>
<br>

# Basic stuff

In [1]:
from typing import Union

import numpy as np
import numpy.linalg as lin
import scipy.optimize
from scipy.linalg import solve as sp_solve

In [2]:
class Problem():

    @staticmethod
    def f(x):
        raise NotImplementedError("f() is not implemented for this Problem")

    @staticmethod
    def grad_f(x):
        raise NotImplementedError("grad_f() is not implemented for this Problem")

    @staticmethod
    def hessian(x):
        raise NotImplementedError("hessian() is not implemented for this Problem")

    @staticmethod
    def equality_constraints(x):
        raise NotImplementedError("equality_constraints() is not implemented for this Problem")

    @staticmethod
    def equalitys_right_hand_side(x):
        raise NotImplementedError("equality_right_hand_side() is not implemented for this Problem")

    @staticmethod
    def inequality_constraints(x):
        raise NotImplementedError("inequality_constraints() is not implemented for this Problem")

    @staticmethod
    def inequalitys_right_hand_side(x):
        raise NotImplementedError("inequality_right_hand_side() is not implemented for this Problem")

    @staticmethod
    def starting_points():
        raise NotImplementedError("starting_points() is not implemented for this Problem")

In [3]:
class Rosenbrock(Problem):

    @staticmethod
    def f(x):
        x = x.reshape((-1, 1))
        return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2

    @staticmethod
    def equality_constraints(x):
       return None

    @staticmethod
    def equalities_right_hand_side(x):
        return None

    @staticmethod
    def inequality_constraints(x):
        x = x.reshape((-1, 1))
        return np.array([-x[0] ** 2 - x[1] ** 2, -x[1]])

    @staticmethod
    def inequalities_right_hand_side(x):
        return np.array([[-1], [0]])

    @staticmethod
    def starting_points():
        return [
            np.array([0.8, 0.6]),
            np.array([1, 0]),
            np.array([1, 1]),
            np.array([0, 0]),
            np.array([-1, 1])
        ]


# this is the function below rosenbrock defined in the project description
class Function2(Problem):

    @staticmethod
    def f(x):
        return 150 * (x[0] * x[1]) ** 2 + (0.5 * x[0] + 2 * x[1] - 2) ** 2

    @staticmethod
    def equality_constraints(x):
       return None

    @staticmethod
    def equalities_right_hand_side(x):
        return None

    @staticmethod
    def inequality_constraints(x):
        x = x.reshape((-1, 1))
        return np.array([(x[0] - (1/2)) ** 2 + (x[1]-1) ** 2 - (5/16),
                         (x[0] + 1) ** 2 + (x[1]-(3/8)) ** 2 - (73/64),
                         (x[0] + 1) ** 2 + (x[1]-1) ** 2 - (np.sqrt(2))])

    @staticmethod
    def inequalities_right_hand_side(x):
        return np.array([[0], [0], [0]])

    @staticmethod
    def starting_points():
        return [
            np.array([0.1, 0.74]),
            np.array([0, 0]),
            np.array([0.5, 0.5]),
            np.array([0, 0.76]),
            np.array([-0.2, 0.8])
        ]

# this is the function below rosenbrock defined in the project description
class Exercise_18_3(Problem):

    @staticmethod
    def f(x):
        return np.exp(np.prod(x)) - ((1/2) * (x[0] ** 3 + x[1] ** 3 + 1) ** 2)

    @staticmethod
    def equality_constraints(x):
       return None

    @staticmethod
    def equalities_right_hand_side(x):
        return None

    @staticmethod
    def inequality_constraints(x):
        x = x.reshape((-1, 1))
        return np.array([(x[0] - (1/2)) ** 2 + (x[1]-1) ** 2 - (5/16),
                         (x[0] + 1) ** 2 + (x[1]-(3/8)) ** 2 - (73/64),
                         (x[0] + 1) ** 2 + (x[1]-1) ** 2 - (np.sqrt(2))])

    @staticmethod
    def inequalities_right_hand_side(x):
        return np.array([[0], [0], [0]])


    @staticmethod
    def starting_points():
        return [
            np.array([-1.71, 1.59, 1.82, -0.763, -0.763]),
            np.array([0, 0, 0, 0, 0]),
            np.array([1, 0, 3, 0, 0])
        ]

# Implementations

## Simplex

In [4]:
def simplex_method(linear_problem: Union[np.ndarray, None], constraints: np.ndarray, constraint_rhs: np.ndarray):
    def swap(variables: list, exits_idx: int, enters_idx: int) -> None:
        """
        In-/excluding respective vectors into/from the basis.
        """
        tmp = variables[exits_idx]
        variables[exits_idx] = variables[enters_idx]
        variables[enters_idx] = tmp
    
    constraint_rhs = constraint_rhs.reshape((-1, 1))
    n_constraints = constraints.shape[0]  # Number of basis vectors
    n_orig_variables = constraints.shape[1]  # Number of original_variables in each constraint
    n_variables = constraints.shape[1]  # Number of variables in each constraint

    tableau = np.concatenate((constraints, np.eye(n_constraints)), axis=1)  # Extended matrix of the system by slack constraints
    x = np.concatenate((np.zeros([n_variables, 1]), constraint_rhs))  # Right parts of the constraints
    
    non_base_weigths = np.zeros((n_variables, 1)) if linear_problem is None else linear_problem
    base_weights = np.ones((n_constraints, 1)) if linear_problem is None else np.zeros((n_constraints, 1))
    weights = np.vstack((non_base_weigths, base_weights))
    
    variable_idx = list(range(len(weights)))  # Indices for all variables
    priv_x_B, x_B = None, None


    while True:
        basis_idx = variable_idx[n_variables:]
        basis = tableau.T[basis_idx].T
        basis_inv = lin.inv(basis)
        priv_x_B = x_B
        x_B = x[basis_idx]
        c_B = weights[basis_idx]
        lam = lin.inv(basis.T) @ c_B

        non_basis_idx = variable_idx[:n_variables]
        c_N = weights[non_basis_idx]
        n_T = tableau.T[non_basis_idx]
        s_N = c_N - (n_T @ lam)

        if np.all(s_N >= 0) or (x_B is not None and priv_x_B is not None and np.sum(priv_x_B - x_B) == 0):
            print("Optimal solution found!!!")
            solution = None if linear_problem is None else c_B.T @ basis_inv @ constraint_rhs
            return x[:n_orig_variables].flatten(), x[n_orig_variables:].flatten(), solution

        # since here must be at least one s_q < 0, we take the argmin to get the index of the smallest value.
        # q_idx = np.argmin(s_N)  # Fetch the idx in the non basis idx set
        # q = non_basis_idx[q_idx]  # Fetch the variable_idx
        # d = basis_inv @ tableau.T[q].T
        # d = np.reshape(d, (-1, 1))

        d = basis_inv @ tableau
        n_q = np.vstack((-d.T[:n_variables].T, np.identity(n_variables)))

        change = (weights.T @ n_q) / np.linalg.norm(n_q, axis=0)
        change[(s_N >= 0).T] = float('inf')
        q_idx = np.argmin(change)
        q = non_basis_idx[q_idx]

        if np.all(d <= 0):
            print("Problem is unbound!!!")
            return None, None, None
        d_q = np.reshape(d[:, q], (-1, 1))
        piv_weigths = x_B / d_q
        p_idx = np.argmin(piv_weigths)
        x_q = piv_weigths[p_idx]
        x[basis_idx] = x_B - (d_q * x_q)
        x[q] = x_q  # set entering variable vaLue

        # In-/excluding respective vectors into/from the basis:
        swap(variables=variable_idx, exits_idx=p_idx + n_variables, enters_idx=q_idx)

In [5]:
# working simplex as placeholder

def simplex(set_b, set_n, A, b, c, print_steps=False):

    A = A.T

    sB = np.zeros((len(set_b), 1))

    xN = np.zeros((len(set_n), 1))

    x = np.zeros((A.shape[0], 1))
    s = np.zeros((A.shape[0], 1))
    
    B = A[set_b].T
    N = A[set_n].T
    cB = c[set_b]
    cN = c[set_n]

    xB = np.linalg.inv(B) @ b

    while True:

        l = np.linalg.inv(B).T @ cB
        
        sN = cN - N.T @ l
        
        if print_steps:
            print(f'\n\nxB:\n{xB}\n\nsN:\n{sN}\n\nlambda:\n{l}\n\nf(x):\n{f(x[:2])}')

        if all(s >= 0 for s in sN):
            optimal_points = [p[0] for p in x]
            return optimal_points

        q = None
        min_change = np.inf
        d = None
        for i, k in enumerate(set_n):

            d_q = np.linalg.inv(B) @ np.reshape(A[k], (-1, 1))
            e = np.identity(c.shape[0] - d_q.shape[0])
            n_q = np.vstack((-d_q, np.reshape(e[i], (-1, 1))))

            change = (c.T @ n_q) / np.linalg.norm(n_q)

            if change < min_change and sN[i] < 0:
                min_change = change
                q = k
                d = d_q

        if all(d_i <= 0 for d_i in d):
            return False

        p = None
        minimum = np.inf
        for i in range(len(d)):
            if d[i] > 0:
                if xB[i] / d[i] < minimum:
                    minimum = xB[i] / d[i]
                    p = i

        x[q] = minimum
              
        xB = xB - d * x[q]  
        x[set_b] = xB[range(len(set_b))]

        xN[set_n.index(q)] = x[q]
        x[set_n] = xN[range(len(set_n))]

        index_n = set_b[p]

        set_b[p] = q
        set_n[set_n.index(q)] = index_n
        
        set_b_len = range(len(set_b))
        set_n_len = range(len(set_n))
        
        B[:, set_b_len] = A[set_b].T
        cB[set_b_len] = c[set_b]
        xB[set_b_len] = x[set_b]
        
        N[:, set_n_len] = A[set_n].T
        cN[set_n_len] = c[set_n]

## Active Set

In [6]:
def compute_learning_rate(A, p, x, w, b):
    numerator = b - (A @ x)
    denominator = A @ p
    possible_alphas = numerator / denominator

    idx_gtz = np.where(denominator > 0)[0].tolist()

    if len(idx_gtz) == 0:
        return 1, None
    else:
        blocking = []
        for idx in idx_gtz:
            if idx not in w:
                blocking.append(idx)

        if len(blocking) == 0:
            return 1, None
        else:
            possible_alphas = possible_alphas[blocking]
            min_alpha = np.argmin(possible_alphas)
            if len(min_alpha.shape) != 0:
                min_alpha = min_alpha[0]

            if possible_alphas[min_alpha] < 1:
                return possible_alphas[min_alpha], min_alpha
            else:
                return 1, None


def compute_descent_direction_and_lagrangien(G, x, c, A, w):
    working_A = A[w]
    size_of_zeros = working_A.shape[0]

#     problem = np.concatenate((
#         np.concatenate((G, working_A.T), axis=1),
#         np.concatenate((working_A, np.zeros((size_of_zeros, size_of_zeros))), axis=1)),
#         axis=0)
    problem = np.vstack((
        np.hstack((G, working_A.T)),
        np.hstack((working_A, np.zeros((size_of_zeros, size_of_zeros))))))
    g_k = (G @ x) + c
    rhs = np.vstack((-g_k, np.zeros((size_of_zeros, 1))))

    result = np.linalg.lstsq(problem, rhs)[0]

    descent_direction = result[0:len(G)]
    lagrangiens = result[len(G):]

    return descent_direction, lagrangiens


def active_set_method(G, c, A, b, inequalities=True, x0=None, steps=1000):
    if x0 is None:
        x, _, _ = simplex_method(None, A, b)
        print(f"No starting point given: Found one feasible point at '{x}'")
    else:
        x = np.copy(x0)
        
    c = c.reshape((-1, 1))
    b = b.reshape((-1, 1))

    # equalities are always active
    w: list = np.where(A @ x == b)[0].tolist() if inequalities else [i for i in range(A.shape[0])]

    for k in range(steps + 1):
        p, lambdas = compute_descent_direction_and_lagrangien(G=G, x=x, c=c, A=A, w=w)

        if np.all(np.abs(p) <= 1e-15):
            if np.all(lambdas >= 0):
                return x, k
            else:
                j = np.argmin(lambdas)
                w_j = w[j]
                while w_j in w:
                    w.remove(w_j)

        else:
            alpha, blocking_idx = compute_learning_rate(A, p, x, w, b)
            x = x + alpha * p

            # Are there blocking constraints
            # http://lendek.net/teaching/OPT/qp.pdf
            if alpha < 1:
                w.append(blocking_idx)

    return None, k

In [7]:
# placeholder asm

def asm(G, W, A, x, c, b, iters=1000, lstsq=False):
    
    c = np.reshape(c, (-1, 1))

    for iteration in range(iters):

        A_k = A[W]
        g_k = G @ x + c

        # construct matrix of linear equations to solve for p_k and lambdas
        """
        form: [G, A.T] [p_k] = [-g_k]
              [A,  0 ] [ l ] = [  0 ]
        """
        top = np.hstack((G, A_k.T))
        bottom = np.hstack((A_k, np.zeros((A_k.shape[0], A_k.shape[0]))))
        big = np.vstack((top, bottom))

        # solve
        equation_vec = np.vstack((-g_k, np.zeros((A_k.shape[0], 1))))
        
        if not lstsq:
            solution = np.linalg.solve(big, equation_vec)
        
        else:
            solution = np.linalg.lstsq(big, equation_vec)[0]

        p_k = solution[:G.shape[0]]

        if all(abs(p_i) < 1e-12 for p_i in p_k):

            l_vec = solution[G.shape[0]:]

            if all(l >= 0 for l in l_vec):
                return x

            else:
                j = np.argmin(l_vec)

                W.pop(j)
        else:

            a_k = 1
            blocking_constraints = []

            for i in range(A.shape[0]):

                if i not in W and A[i].T @ p_k > 0:
                    candidate = (b[i] - A[i] @ x) / (A[i] @ p_k)
                    if candidate <= a_k:
                        a_k = candidate
                        blocking_constraints.append(i)

            x = x + a_k * p_k

            if a_k < 1:
                W += [blocking_constraints.pop()]

## SQP

In [8]:
def e_i(size, index):
    arr = np.zeros((size, 1))
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        x= x.reshape((-1, 1))
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        x= x.reshape((-1, 1))
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f


def approx_hessian_bfgs(Bk, f, Aieq, Aeq, x, x1, lam, pk, alpha):
    """
    Approximate Hessian based on first derivitive using previous values.
    :param f_grad: first derivitive of f
    :param B_k: current hessian
    :param x_1: x_{k+1}
    :param x: x_{k}
    :return: approximated hessian
    """
    def L(x):
        constraints = None
        if Aieq is not None and Aieq(x) is not None:
            constraints = Aieq(x)
        if Aeq is not None and Aeq(x) is not None:
            if constraints is None:
                constraints = Aeq(x)
            else:
                constraints = np.vstack((constraints, Aeq(x)))

        res = lam.T @ constraints
        res += f(x)

        return res

    grad_langrange = approx_grad(L)
    x1 = x1.reshape((-1, 1))
    x = x.reshape((-1, 1))

    s = alpha * pk
    y = grad_langrange(x1) - grad_langrange(x)
    y = y.reshape((-1, 1))
    o = 1 if s.T @ y >= .2 * s.T @ Bk @ s else (.8 * s.T @ Bk @ s) / (s.T @ Bk @ s - s.T @ y)
    y = o * y + (1 - o) * Bk @ s

    rho_inv = np.dot(y.T, s)
    if rho_inv == 0:
        rho_inv = 1e-4
    hs = (s.T.dot(Bk.dot(s)))
    if hs == 0:
        hs = 1e-4

    Bk1 = Bk - (Bk @ np.outer(s, s) @ Bk) / hs + (np.outer(y, y) / rho_inv)
    return Bk1


def compute_lagrangien_and_decent_sqp(H, x, eq_constraints, ieq_constraints, dfx):
    A = None
    c = None
    if ieq_constraints is not None and ieq_constraints(x) is not None:
        ieq_grad = approx_grad(f=ieq_constraints)
        A = ieq_grad(x)
        c = ieq_constraints(x)
    if eq_constraints is not None and eq_constraints(x) is not None:
        eq_grad = approx_grad(f=eq_constraints)
        if A is None:
            A = eq_grad(x)
        else:
            A = np.vstack((A, eq_grad(x)))

        if c is None:
            c = eq_constraints(x)
        else:
            c = np.vstack((c, eq_constraints(x)))

    A = A.squeeze()
    n_constraints = len(c)
    zeros = np.zeros((n_constraints, n_constraints))

    descent_problem = np.vstack((H, A.T))
    langragian_problem = np.vstack((-A, zeros))
    lhs = np.hstack((descent_problem, langragian_problem))
    rhs = np.vstack((-dfx, -c))

    res = sp_solve(lhs, rhs)
    n_var = len(x)

    return res[:n_var], res[n_var:]


def choose_mu(H, fpk, pk, ck, mu=None, rho=0.8, mu_delta=0, sigma=1):
    nominator = fpk.T @ pk + (sigma / 2) * pk.T @ H @ pk
    denominator = (1 - rho) * np.linalg.norm(ck, 1)

    lower_bound = nominator / denominator

    if mu is None or mu < lower_bound:
        mu = lower_bound + mu_delta

    return mu


def l1_merit_function(f, x, Aieq, Aeq, mu):
    fx = f(x)

    constraints = None
    if Aieq is not None and Aieq(x) is not None:
        constraints = Aieq(x)
    if Aeq is not None and Aeq(x) is not None:
        if constraints is None:
            constraints = Aeq(x)
        else:
            constraints = np.vstack((constraints, Aeq(x)))

    return (fx + mu * np.sum(np.maximum(-constraints, 0))).squeeze()


def choose_alpha(f, fpk, x, ck, pk, Aieq, Aeq, eta, mu, tau):
    alpha = 1
    x = x.reshape((-1, 1))

    while True:
        compare_value = x + (alpha * pk)
        compare_merit_value = l1_merit_function(f, compare_value, Aieq, Aeq, mu)
        base_merit = l1_merit_function(f, x, Aieq, Aeq,  mu)
        directional_derivate = fpk.T @ pk - mu * np.linalg.norm(ck, ord=1)

        base = base_merit + eta * alpha * directional_derivate

        if compare_merit_value <= base:
            return alpha

        tau_alpha = np.random.uniform(low=0, high=tau)
        alpha = alpha * tau_alpha


def stop_crit(gradient, ieq_constraints, eq_constraints, xk, x0, lamk, current_iteration, tol: float = 1e-8, max_iter: int = 10000):
    A = None
    c_i = None
    c_e = None
    len_c_i = 0
    if ieq_constraints is not None and ieq_constraints(xk) is not None:
        ieq_grad = approx_grad(f=ieq_constraints)
        A = ieq_grad(xk)
        c_i = ieq_constraints(xk)
        len_c_i = len(c_i)
    if eq_constraints is not None and eq_constraints(xk) is not None:
        eq_grad = approx_grad(f=eq_constraints)
        if A is None:
            A = eq_grad(xk)
        else:
            A = np.vstack((A, eq_grad(xk)))

        c_e = eq_constraints(xk)

    A = A.squeeze()
    dfxk = gradient(xk)
    lam_i = lamk[:len_c_i]

    result = True

    # KKT Conditions must be all met.
    result = (result and np.all(np.abs(dfxk + A @ lamk) <= 1e-10))
    result = (result and np.all(np.abs(c_e) <= 1e-10))
    result = (result and np.all(c_i <= 0))
    result = (result and np.all(lam_i >= 0))
    result = (result and np.all(np.abs((lam_i.T @ c_i) <= 1e-10)))

    # Or Maximum iterations reached
    result = (result or (current_iteration > max_iter))

    # Or point has converged to a point
    if not result:
        result = np.linalg.norm(gradient(xk)) <= tol * np.linalg.norm(gradient(x0))

    return result


def sqp_method(f, Aieq=None, bieq= None, Aeq=None, beq=None, x0=None, lam0=None, steps=1000,
               mu_delta: float = 1e-2, eta: float = 0.3, tau: float = 0.9):

    # Initialize values
    gradient, hessian = approx_grad(f=f), approx_hessian(f=f)
    xk = x0.astype(float)

    lamk, muk = None, None
    pk, pl = None, None

    iteration = 1

    while True:
        dfk = gradient(xk).reshape((-1, 1))
        ddfk = hessian(xk).squeeze()

        constraints = None
        if Aieq is not None and Aieq(xk) is not None:
            constraints = Aieq(xk)
        if Aeq is not None and Aeq(xk) is not None:
            if constraints is None:
                constraints = Aeq(xk)
            else:
                constraints = np.vstack((constraints, Aeq(xk)))

        if lamk is None:
            lamk = np.ones((len(constraints), 1)) if lam0 is None else lam0.astype(float)
        if muk is None:
            muk = np.max(lamk) + mu_delta

        # stopping condition
        if iteration > 1:
            if stop_crit(gradient, Aieq, Aeq, xk, x0, lamk, iteration, max_iter=steps):
                print("Found solution!!!!")
                return xk, iteration

        # Take a step
        pk, lamh = compute_lagrangien_and_decent_sqp(H=ddfk, x=xk, eq_constraints=Aeq, ieq_constraints=Aieq, dfx=dfk)
        pl = lamh if pl is None else (pl - lamh)
        muk = choose_mu(ddfk, dfk, pk, constraints, mu=muk, mu_delta=mu_delta)
        alpha_k = choose_alpha(f=f, fpk=dfk, x=xk, ck=constraints, pk=pk, Aieq=Aieq, Aeq=Aeq,
                               eta=eta, mu=muk, tau=tau)

        xk1 = xk.reshape((-1, 1)) + alpha_k * pk
        lamk1 = lamk + alpha_k * pl
        ddfk = approx_hessian_bfgs(ddfk, f, Aieq, Aeq, xk, xk1, lamk1, pk, alpha_k)
        xk, lamk = xk1, lamk1
        iteration += 1

## Derivative Approximations

In [9]:
def e_i(size, index):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f

# Testruns

## Simplex

In [10]:
prob = np.array([[-3], [-2]])
const = np.array([[1, 1], [2, 0.5]])
rhs = np.array([[5], [8]])

x_val, slack_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
print(f"x = {x_val}")
print(f"slack = {slack_val}")
print(f"solution = {sol}")

Optimal solution found!!!
x = [3.66666667 1.33333333]
slack = [0. 0.]
solution = [[-13.66666667]]


## Generating and solving simplex problems

For the simplex method run 2 chooses an infeasible basis. Run 1 should therefore always give the correct minimum.

In [11]:
def generate_problem(n_variables, n_constraints):
    A = np.random.randint(-10, 11, size=(n_constraints, n_variables))
    A = np.concatenate((A, np.identity(n_constraints)), axis=1)
    c = np.random.randint(-10, 11, size=(n_variables, 1))
    c = np.vstack((c, np.zeros((n_constraints, 1))))
    b = np.random.randint(0, 11, size=(n_constraints, 1))

    set_b = [i for i in range(A.shape[1] - n_constraints, A.shape[1])]
    set_n = [i for i in range(A.shape[1] - n_constraints)]

    return A, b, c, set_b, set_n

In [12]:
def generate_and_solve(n_problems, n_variables, n_constraints):
    for i in range(n_problems):
        print(f'PROBLEM {i+1}, RUN 1')

        A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)

        result = simplex(set_b, set_n, A, b, c)

        while result == False or np.sum(result) < 1e-15:
            A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)
            result = simplex(set_b, set_n, A, b, c)

        print(f'A:\n{A}\nb:\n{b}\nc:\n{c}\n')
        print(f'Minimum at: {result}\n\n')
        
        print(f'PROBLEM {i+1}, RUN 2')
        
        set_b[-1], set_n[-1] = set_n[-1], set_b[-1]

        result = simplex(set_b, set_n, A, b, c)

        print(f'A:\n{A}\nb:\n{b}\nc:\n{c}\n')
        print(f'Minimum at: {result}\n\n')

In [13]:
generate_and_solve(n_problems=5, n_variables=10, n_constraints=5)

PROBLEM 1, RUN 1
A:
[[  8.  -3. -10.  -6.  -2. -10.  -2.  -8.  -2.  -1.   1.   0.   0.   0.
    0.]
 [  3.   8.   8.   7.  -8.   5.   9.   8.   9.   4.   0.   1.   0.   0.
    0.]
 [  6.   6.  -5.   3.  -2.   1.  10.   2.  -5.   5.   0.   0.   1.   0.
    0.]
 [  1.   6.   7.  -9.   8.  -1.  -8.  -2.   3.  -6.   0.   0.   0.   1.
    0.]
 [  8.  -6.   6.   8.  -9.  10.   5.  -8.   1.   3.   0.   0.   0.   0.
    1.]]
b:
[[1]
 [6]
 [3]
 [2]
 [9]]
c:
[[ 6.]
 [-9.]
 [ 1.]
 [-1.]
 [ 3.]
 [ 1.]
 [-8.]
 [ 3.]
 [ 9.]
 [ 7.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

Minimum at: [0.0, 0.4913522012578616, 0.09905660377358497, 0.18238993710691823, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.558962264150944, 0.19025157232704407, 0.3, 0.40740740740740744, 9.894654088050315]


PROBLEM 1, RUN 2
A:
[[  8.  -3. -10.  -6.  -2. -10.  -2.  -8.  -2.  -1.   1.   0.   0.   0.
    0.]
 [  3.   8.   8.   7.  -8.   5.   9.   8.   9.   4.   0.   1.   0.   0.
    0.]
 [  6.   6.  -5.   3.  -2.   1.  10.   2.  -5.   5.   0.   0.  

## Active Set

In [14]:
G = np.array([[2, 0, 0, 0, 0],
              [0, 2, 0, 0, 0],
              [0, 0, 2, 0, 0],
              [0, 0, 0, 2, 0],
              [0, 0, 0, 0, 2]])
c = np.array([-4, -4, 0, 0, 5])
A = np.array([[1, 1, 0, 0, 0], [0, 0, 1, -2, 0]])
b = np.array([[2], [2]])

# feasible_x, W = find_feasible_point(A, b, n_constraints=3, n_variables=2)
# print(feasible_x, W)

# result, iterations = active_set_method(G, c, A, b)
# print(f"Found solution at {result} after {iterations} iterations")

## Generating and solving QP problems

Regarding the linear constraints: Here they are randomly generated. This is due to us being unsure as to how to construct them from the definition given in the assignment.
<br><br>
Our interpretation was that to make the constraints smooth and linear we would have to split them up as follows:
<br><br>
$|x_1| + |x_2| \leq 1 \implies x_1 + x_2 \leq 1, x_1 - x_2 \leq 1, -x_1 + x_2 \leq 1, -x_1 - x_2 \leq 1$
<br><br>
For twenty variables however, as described in Problem that would lead to $2^20$ constraints, which seems a bit unreasonable

In [15]:
def generate_and_solve_qp(m_rows=5, M=None, y=None, x0=None, use_lstsq=False):
    
    if M is None:
        M = np.random.randint(-10, 10, size=(m_rows, 2*m_rows))
    
    if y is None:
        y = np.random.randint(-10, 10, size=(m_rows, 1))
    
    G = M.T @ M
    c = -(M.T @ y)
    b = np.ones((M.shape[1], 1), dtype=int)
    
    # for now random constraints
    
    A = np.random.randint(-20, 21, size=(b.shape[0], G.shape[0]))

    n_constraints = b.shape[0]
    n_variables = G.shape[0]
    
    A_simp = np.hstack((A, np.identity(n_constraints)))
    c_simp = np.ones((n_constraints, 1))
    c_simp = np.vstack((np.zeros((n_variables, 1)), c_simp))

    set_b = [i for i in range(A_simp.shape[1] - n_constraints, A_simp.shape[1])]
    set_n = [i for i in range(A_simp.shape[1] - n_constraints)]

    minimiser = simplex(set_b, set_n, A_simp, b, c_simp)

    x = np.reshape(minimiser[:n_variables], (-1, 1))

    W = np.where(A @ x == b)[0].tolist()
    
    result = asm(G, W, A, x, c, b, lstsq=use_lstsq)

    return W, G, A, b, c, x, result

## 3 runs of QP Problem 1

In [16]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 0 attempts.
W:
[4, 3, 7, 9, 2]
G:
[[ 239  115 -216  -74   -6    7 -100   18   19   17]
 [ 115  172  -37  -18   52  -30  -80   55   18    1]
 [-216  -37  289  140  128   35  100   61  -55   20]
 [ -74  -18  140  123  109   18   12  137  -78   35]
 [  -6   52  128  109  262  113   48   72  -90  187]
 [   7  -30   35   18  113  155  116  -94  -14   83]
 [-100  -80  100   12   48  116  147 -135   10   30]
 [  18   55   61  137   72  -94 -135  306 -103   -3]
 [  19   18  -55  -78  -90  -14   10 -103   68  -67]
 [  17    1   20   35  187   83   30   -3  -67  214]]
A:
[[ 13  -7 -13 -18 -15   9 -17  -7  -3 -19]
 [  5  19  12   8   3   4 -11  16 -19   3]
 [  2 -20 -15  -2  14 -17   4  20  12  -1]
 [ 19 -14  -8 -19  -8  -6  14 -15   0  -3]
 [ 19 -19  -8  17  15  20  20  16   7   7]
 [ 14  15  11   6  -7   8 -14 -13 -19  10]
 [ -9 -13 -16 -15   4  -7   5  14   3 -14]
 [-13   0  15  16  19 -13 -20  -4   2  -4]
 [ -3  -7  12  18  11  16   6  12 -13 -20]
 [ 20 -20  14 -

## 3 runs of QP Problem 2

In [17]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 1 attempts.
W:
[8, 6, 2, 5, 3, 0]
G:
[[ 193  -25   60  -66  -13  100   44  -95    1  -26]
 [ -25  282   50  -87 -136 -121 -145   36  102 -143]
 [  60   50  120   44   12   16   54   16  -54   18]
 [ -66  -87   44  136  100   20   97   66 -110  110]
 [ -13 -136   12  100  127  108   89   31 -106   87]
 [ 100 -121   16   20  108  188   71  -26  -76   26]
 [  44 -145   54   97   89   71  191   19 -120  136]
 [ -95   36   16   66   31  -26   19   87  -42   29]
 [   1  102  -54 -110 -106  -76 -120  -42  115 -100]
 [ -26 -143   18  110   87   26  136   29 -100  130]]
A:
[[ 20 -16   6  -6  -6   4   6  15   1 -10]
 [ 11  12  -7  17   1  16  13   2  -4  -1]
 [ 19 -17   2  -1  10 -12 -20   7  -5 -12]
 [  1   4 -14  20   0   2  -7 -20   8  -7]
 [-14  12   4   1  -9  -5  16  -8  -6  -3]
 [ -8   7  12   6 -10  -4  -8 -17   5  -2]
 [ 10  12  17   7 -20  -2 -16  11 -18   6]
 [-14  10  -5  -6  12 -16  17 -19  14 -19]
 [-11  19  19  18  15 -12   4 -17   0  20]
 [ -3   3  1

## 4 runs of QP Problem 3

In [18]:
zero_matrix = np.zeros((2, 4), dtype=int)
M_hat = np.array([[1, 1, 0, 0], [0, 0, 1, 1]])

M_1 = np.hstack((M_hat, zero_matrix, zero_matrix, zero_matrix, zero_matrix))
M_2 = np.hstack((zero_matrix, M_hat, zero_matrix, zero_matrix, zero_matrix))
M_3 = np.hstack((zero_matrix, zero_matrix, M_hat, zero_matrix, zero_matrix))
M_4 = np.hstack((zero_matrix, zero_matrix, zero_matrix, M_hat, zero_matrix))
M_5 = np.hstack((zero_matrix, zero_matrix, zero_matrix, zero_matrix, M_hat))

M = np.vstack((M_1, M_2, M_3, M_4, M_5))
print((M).shape)

y = np.reshape([1, -2, 3, -4, 5, -5, 4, -3, 2, -1], (-1, 1))

W, G, A, b, c, x, result = generate_and_solve_qp(M=M, y=y, use_lstsq=True)

print('\n\nRUN 1\n\n')
print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

print('\n\nRUN 2\n\n')
x = np.reshape([0]*20, (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters, lstsq=True)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

print('\n\nRUN 3\n\n')
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters, lstsq=True)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')
    
print('\n\nRUN 4\n\n')
x = np.reshape([1]*20, (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters, lstsq=True)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

(10, 20)


RUN 1


Feasible function found after 1 attempts.
W:
[1, 17, 0, 13, 18, 5, 3, 11, 7, 15, 9]
G:
[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
A:
[[ 20   5 -14  -4  -7  11  10 

C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\192508625.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  solution = np.linalg.lstsq(big, equation_vec)[0]


## SQP

In [19]:
problems = [Rosenbrock, Exercise_18_3, Function2]


for prob in problems:
    print(f"Function {type(prob.__class__)}")

    for sp in prob.starting_points():
        print(f"Use '{sp}' as starting point")

        try:
            result, iteration = sqp_method(prob.f,
                                           prob.inequality_constraints, prob.inequalities_right_hand_side,
                                           prob.equality_constraints, prob.equalities_right_hand_side,
                                           x0=sp)
            print(f"After {iteration} iterations we got point '{result}'")
        except numpy.linalg.LinAlgError as e:
            print(e)

        print("-------------------------------------------------------")

print("========================================================================")

Function <class 'type'>
Use '[0.8 0.6]' as starting point


C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=1.51663e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.22079e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.35585e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=3.01642e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.98146e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-con

C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=3.07042e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.8721e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.9377e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=2.91626e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=3.12639e-21): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-condi

Found solution!!!!
After 1001 iterations we got point '[[0.60055841]
 [0.0244416 ]]'
-------------------------------------------------------
Use '[1 0]' as starting point
Found solution!!!!
After 1001 iterations we got point '[[5.00000006e-01]
 [1.25458205e-13]]'
-------------------------------------------------------
Use '[1 1]' as starting point


C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=1.72286e-22): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=1.95856e-22): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=3.53987e-22): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=5.66167e-22): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-conditioned matrix (rcond=7.27786e-22): result may not be accurate.
  res = sp_solve(lhs, rhs)
C:\Users\weikl\AppData\Local\Temp\ipykernel_32400\607868026.py:103: LinAlgWarning: Ill-con

Found solution!!!!
After 1001 iterations we got point '[[0.84673524]
 [0.15326476]]'
-------------------------------------------------------
Use '[0 0]' as starting point


NameError: name 'numpy' is not defined